# 🚀 Proyecto ELT con SpaceX API  
## Notebook 02 – Transformación de datos

En este notebook se aplican tareas de **procesamiento y transformación de datos con Pandas**, según la consigna:  

- Eliminación de duplicados.  
- Manejo de valores nulos.  
- Renombrado de columnas.  
- Conversión de tipos.  
- Creación de nuevas columnas.  
- Unión (JOIN) entre DataFrames.  


In [ ]:
# --- IMPORTS ---
from src.extract import fetch_data
from src.transform import drop_duplicates, handle_nulls, rename_columns, convert_column_types, create_new_column, join_dataframes
from src.utils import setup_logger

# --- LOGGER ---
logger = setup_logger()
logger.info("Inicio de transformaciones de datos")


## 📥 Lectura de datos

Se reextraen los datasets desde la API (también se podrían leer desde Delta Lake).  
Trabajaremos principalmente con:  
- **Próximos lanzamientos** (`upcoming_launches`)  
- **Cohetes** (`rockets`)  


In [ ]:
# Extracción de datos a transformar
df_upcoming = fetch_data("upcoming_launches")
df_rockets = fetch_data("rockets")


## 🛠 Transformaciones aplicadas
1. **Eliminar duplicados**.  
2. **Rellenar valores nulos** en la columna `details`.  
3. **Renombrar columnas** para mayor claridad (`date_utc → fecha_lanzamiento`).  
4. **Conversión de tipos** de fecha.  
5. **Nueva columna**: indicador booleano si el lanzamiento es en 2025.  
6. **Join** entre lanzamientos y cohetes.  


In [ ]:
# 1. Eliminar duplicados
df_upcoming = drop_duplicates(df_upcoming)

# 2. Manejo de nulos
df_upcoming = handle_nulls(df_upcoming, cols=["details"], fill_value="Sin descripción")

# 3. Renombrar columnas
df_upcoming = rename_columns(df_upcoming, {"date_utc": "fecha_lanzamiento"})

# 4. Conversión de tipos
df_upcoming = convert_column_types(df_upcoming, {"fecha_lanzamiento": "datetime64[ns]"})

# 5. Nueva columna: ¿lanzamiento en 2025?
df_upcoming = create_new_column(
    df_upcoming,
    "es_2025",
    lambda row: row["fecha_lanzamiento"].year == 2025 if row["fecha_lanzamiento"] is not pd.NaT else False
)

# 6. Join con info de cohetes
df_final = join_dataframes(df_upcoming, df_rockets, key="rocket")

logger.info(f"Transformaciones aplicadas. Total de registros finales: {len(df_final)}")


## 👀 Vista de resultados

Mostramos algunas columnas seleccionadas del DataFrame final:  


In [ ]:
df_final[["name_x", "fecha_lanzamiento", "es_2025", "name_y", "active"]].head()
